In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
from time import time

In [2]:
test = spark.read.\
                option("sep", ";").\
                option("header", "true").\
                csv("/user/datacovid/HIST_PAINEL_COVIDBR_*.csv")

In [ ]:
# Selecionando no dataFrame somente os municipios não nulos
municipios = test.filter(test.municipio.isNotNull()).distinct()

In [ ]:
# Separando em um novo dataframe somente os Municipios
mc = municipios.select(municipios.municipio).distinct()

### Criando o Database no HIVE

In [ ]:
sqlContext.sql("CREATE DATABASE covid location '/user/hive/warehouse/covid19'")

In [3]:
# Verificando a criação Banco covid19
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|       banco|
|       covid|
|     covid19|
|     default|
+------------+



In [4]:
sqlContext.sql("use covid")

DataFrame[]

In [ ]:
sqlContext.sql("CREATE TABLE test(regiao string, estado string, municipio string, codUf string, codMun string, codRegiaoSaude string, nomeRegiaoSaude string, data string, semanaEpi string, populacaoTCU2019 string, casosAcumulado string, casosNovos string, obitosAcumulado string, obitosNovos string, recuperadosNovos string, emAcompanhamentoNovos string, interiorMetropolitana string)")

### Particionando os arquivos por municipio

In [ ]:
# Loop para construção de uma lista de municípios
list_municipios  = []
for row in mc.collect():
    list_municipios.append(row.municipio)

In [ ]:
# Loop para separar os dados do DF por municipio, assim otimizar e particionar por municipio, assim como pede no exercicio.
for data in list_municipios:
    df = test.filter(test.municipio.isin(data))
    table = data.replace(" ", "_")
    table = table.lower()
    # sqlContext.sql("ALTER TABLE test ADD PARTITION(municipio="+ data +"))
    df.write.saveAsTable(table)

In [ ]:
## Códigos aleatórios verificar se necessário
test.filter(test.municipio.isin(municipios.municipio)).show()
test.createOrReplaceTempView("COVIDBR_2020_Parte1")
df = spark.sql("SELECT * FROM COVIDBR_2020_Parte1 where municipio <> 'null'")

### Montar as 3 vizualizações

* | Casos Recuperados, Em acompanhamento | 17.262.646, 1.065.477
* | Casos confirmados ( Acumulado, casos novos, Incidência) | 18.855.015, 62.504, 8972,3
* | Óbitos confirmados ( Acumulado, casos novos, letalidade, mortalidade) | 526.892, 1.780, 2,8%, 250,7


In [ ]:
# Criando a tabela com todos os dados
test.write.partitionBy(test.municipio).parquet()

In [ ]:
# Criar Visualização de Recuperados e Em acompanhamento

In [ ]:
x = datetime.now()
print(x)

In [ ]:
y = datetime.now()
print(y-x)


In [ ]:
# Casos Recuperados, Em acompanhamento

x = datetime.now()

Casos_Recuperados_DF = \
    sqlContext.sql( \
        "SELECT \
            FORMAT_NUMBER(SUM(Recuperadosnovos),0) AS Casos_Recuperados, \
            FORMAT_NUMBER(SUM(emAcompanhamentoNovos),0) AS Em_Acompanhamento \
        FROM \
            alldata \
        WHERE \
            data = '2021-07-06'").show()

y = datetime.now()
print("TEMPO DE EXECUÇÃO -->", y-x)

In [22]:
# Casos confirmados

Casos_Confirmados_DF = \
    spark.sql( \
        "SELECT \
            FORMAT_NUMBER((SUM(Recuperadosnovos) + SUM(emAcompanhamentoNovos) + SUM(obitosAcumulado)),0) AS Acumulado, \
            (\
                SELECT \
                    FORMAT_NUMBER(SUM(casosNovos),0) \
                FROM \
                    alldata \
                WHERE \
                    data = '2021-07-05' \
             ) AS Casos_Novos, \
            FORMAT_NUMBER(((SUM(Recuperadosnovos) + SUM(emAcompanhamentoNovos) + SUM(obitosAcumulado)) * 100000) / SUM(populacaoTCU2019), 0) as Incidencia \
        FROM \
            alldata \
        WHERE \
            data = '2021-07-06'").show()

+----------+-----------+----------+
| Acumulado|Casos_Novos|Incidencia|
+----------+-----------+----------+
|19,908,799|     68,109|     3,158|
+----------+-----------+----------+



In [21]:
# Óbitos Confirmados

Casos_Obtos_DF = \
    spark.sql( \
        "SELECT \
            FORMAT_NUMBER(((SUM(obitosAcumulado))),0) AS Obitos_acumulados, \
            (\
                SELECT \
                    FORMAT_NUMBER(SUM(obitosNovos),0) \
                FROM \
                    alldata \
                WHERE \
                    data = '2021-07-05' \
            ) AS Obitos_novos, \
            ROUND(((SUM(obitosAcumulado)) * 100) / (SUM(Recuperadosnovos) + SUM(emAcompanhamentoNovos) + (SUM(obitosAcumulado))),2) AS Letalidade, \
            ROUND(((((SUM(obitosAcumulado))) * 100000)) / SUM(populacaoTCU2019), 2) AS Mortalidade \
        FROM \
            alldata \
        WHERE data = '2021-07-06'").show()

+-----------------+------------+----------+-----------+
|Obitos_acumulados|Obitos_novos|Letalidade|Mortalidade|
+-----------------+------------+----------+-----------+
|        1,580,676|       2,085|      7.94|     250.73|
+-----------------+------------+----------+-----------+



In [ ]:
# Salvando Casos Recuperados, Em acompanhamento em tabela Hive

Casos_Recuperados_DF.write.saveAsTable("casos")

In [ ]:
# Lendo a tabela casos

sqlContext.sql("SELECT * casos").show()

In [ ]:
Casos_Confirmados_DF.write.mode("overwrite").option("compression", "snappy").parquet("/user/datacovid/")